# Analysis of deep mutational scanning of influenza A/Aichi/1968 (H3N2) nucleoprotein in perturbed host proteostasis environments

Anna Ponomarenko, March 2018

## Load required modules

In [ ]:
import os
import glob
import sys
import string
import time
import multiprocessing

import pandas as pd

basedir = os.getcwd()
if not os.path.isdir('notebook_figures'):
    !mkdir notebook_figures
if not os.path.isdir('FASTQ_files'):
    !mkdir FASTQ_files
if not os.path.isdir('preferences'):
    !mkdir preferences
    
import matplotlib
print "Using matplotlib version %s" % matplotlib.__version__
matplotlib.use("Pdf")
orange = "#E69F00"
lightblue = "#56B4E9"
green = "#009E73"
yellow = "#F0E442"
darkblue = "#0072B2"
rust =  "#D55E00"
purple =  "#CC79A7"
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Polygon
from matplotlib import gridspec
import pylab as plt

from IPython.display import Image, display

import dms_tools
print "Using dms_tools version %s" % dms_tools.__version__
import dms_tools.utils
import dms_tools.file_io
import dms_tools.plot

### Define some functions and variables used below by multiple cells

In [ ]:
def RunScriptDms(rundir, run_name, script_name, commands, use_sbatch, sbatch_cpus, walltime=None):
    """Runs a ``dmstools`` script.

    *rundir* is the directory in which we run the job. Created if it does
    not exist.

    *run_name* is the name of the run, which should be a string without
    spaces. The input file has this prefix followed by ``_infile.txt``.

    *script_name* is the name of the script that we run.

    *commands* is a list of command-line arguments. These arguments should appear 
    in the order in which one would place them on the command line. The elements in this
    list are joined together by spaces, and given to *script_name*. All elements
    should be strings.

    *use_sbatch* is a Boolean switch specifying whether we use ``sbatch``
    to run the script. If *False*, the script is just run with the command
    line instruction. If *True*, then ``sbatch`` is used, and the command file
    has the prefix *run_name* followed by the suffix ``.sbatch``.

    *sbatch_cpus* is an option that is only meaningful if *use_sbatch* is 
    *True*. It gives the integer number of CPUs that are claimed via
    ``sbatch`` using the option ``sbatch -c``. 

    *walltime* is an option that is only meaningful if *use_sbatch* is
    *True*. If so, it should be an integer giving the number of hours 
    to allocate for the job. If *walltime* has its default value of 
    *None*, no wall time for the job is specified.

    It is assumed that the script can be run at the command line using::

        script_name infile

    Returns *runfailed*: *True* if run failed, and *False* otherwise.
    """
    print "Running %s for %s in directory %s..." % (script_name, run_name, rundir)
    currdir = os.getcwd()
    if not os.path.isdir(rundir):
        os.mkdir(rundir)
    os.chdir(rundir)
    if (not run_name) or not all([x not in string.whitespace for x in run_name]):
        raise ValueError("Invalid run_name of %s" % run_name)
    commandline = ' '.join(['%s' % command for command in commands])
    if use_sbatch:
        sbatchfile = '%s.sbatch' % run_name # sbatch command file
        jobidfile = 'sbatch_%s_jobid' % run_name # holds sbatch job id
        jobstatusfile = 'sbatch_%s_jobstatus' % run_name # holds sbatch job status
        joberrorsfile = 'sbatch_%s_errors' % run_name # holds sbatch job errors
        sbatch_f = open(sbatchfile, 'w')
        sbatch_f.write('#!/bin/sh\n#SBATCH\n')
        if walltime:
            sbatch_f.write('#PBS -l walltime=%d:00:00\n' % walltime)
        sbatch_f.write('%s %s' % (script_name, commandline))
        sbatch_f.close()
        os.system('sbatch -c %d -e %s %s > %s' % (sbatch_cpus, joberrorsfile, sbatchfile, jobidfile))
        time.sleep(60) # short 1 minute delay
        jobid = int(open(jobidfile).read().split()[-1])
        nslurmfails = 0
        while True:
            time.sleep(60) # delay 1 minute
            returncode = os.system('squeue -j %d > %s' % (jobid, jobstatusfile))
            if returncode != 0:
                nslurmfails += 1
                if nslurmfails > 180: # error if squeue fails at least 180 consecutive times
                    raise ValueError("squeue is continually failing, which means that slurm is not working on your system. Note that although this script has crashed, many of the jobs submitted via slurm may still be running. You'll want to monitor (squeue) or kill them (scancel) -- unfortunately you can't do that until slurm starts working again.")
                continue # we got an error while trying to run squeue
            nslurmfails = 0
            lines = open(jobstatusfile).readlines()
            if len(lines) < 2:
                break # no longer in slurm queue
        errors = open(joberrorsfile).read().strip()
    else:
        errors = os.system('%s %s' % (script_name, commandline))
    os.chdir(currdir)
    if errors:
        print "ERROR running %s for %s in directory %s." % (script_name, run_name, rundir)
        return True
    else:
        print "Successfully completed running %s for %s in directory %s." % (script_name, run_name, rundir)
        return False

    
def RunProcesses(processes, nmultiruns):
    """Runs a list *multiprocessing.Process* processes.

    *processes* is a list of *multiprocessing.Process* objects that
    have not yet been started. If an empty list, then just returns 
    with no action taken.

    *nmultiruns* is an integer >= 1 indicating the number of simultaneous
    processes to run.

    Runs the processes in *processes*, making sure to never have more than
    *nmultiruns* running at a time. If any of the processes fail (return
    an exitcode with a boolean value other than *False*), an exception
    is raised immediately. Otherwise, this function finishes when all
    processes have completed.
    """
    if not processes:
        return
    if not (nmultiruns >= 1 and isinstance(nmultiruns, int)):
        raise ValueError("nmultiruns must be an integer >= 1")
    processes_started = [False] * len(processes)
    processes_running = [False] * len(processes)
    processes_finished = [False] * len(processes)
    while not all(processes_finished):
        if (processes_running.count(True) < nmultiruns) and not all(processes_started):
            i = processes_started.index(False)
            processes[i].start()
            processes_started[i] = True
            processes_running[i] = True
        for i in range(len(processes)):
            if processes_running[i]:
                if not processes[i].is_alive():
                    processes_running[i] = False
                    processes_finished[i] = True
                    if processes[i].exitcode:
                        raise IOError("One of the processes failed to complete.")
        time.sleep(60)

max_cpus = 1
# Do we use sbatch when appropriate? Some simple operations will still
# be run without sbatch even when True
use_sbatch = False
# Specify reference sequence to map reads to, numbering must follow what is specified by alignspecs
refseq = '%s/Aichi_NP_reference.fasta' % basedir

# Specify the biological replicates and sample names. Sample names are stored as list of 2-tuples or in a dictionary
replicates = ['replicate-1', 'replicate-2']
replicate_sample = [('replicate-1', 'plasmid'),('replicate-1', 'p0mut'),('replicate-1', 'WT37'),('replicate-1', 'DMSO37'),('replicate-1', 'dnHSF137'), ('replicate-1', '17AAG37'), ('replicate-1', 'PU29F37'),('replicate-1', 'WT39'),('replicate-1', 'DMSO39'),('replicate-1', 'dnHSF139'), ('replicate-1', '17AAG39'), ('replicate-1', 'PU29F39'),('replicate-2', 'plasmid'),('replicate-2', 'p0mut'),('replicate-2', 'WT37'),('replicate-2', 'DMSO37'),('replicate-2', 'dnHSF137'), ('replicate-2', '17AAG37'), ('replicate-2', 'PU29F37'),('replicate-2', 'WT39'),('replicate-2', 'DMSO39'),('replicate-2', 'dnHSF139'), ('replicate-2', '17AAG39'), ('replicate-2', 'PU29F39')]
#, ('replicate-1', 'plasmid'),('replicate-1', 'WTplasmid')]
replicate_sample_dict = dict([ (replicate, []) for replicate in replicates])
for replicate, sample in replicate_sample:
    replicate_sample_dict[replicate].append(sample)
        
print("Shared functions and variables are now defined.")

### Create reference file

In [ ]:
%%writefile $refseq
>gi|392340231|gb|CY121120.1| Influenza A virus (A/Aichi/2/1968(H3N2)) nucleocapsid protein (NP) gene, complete cds
ATGGCGTCCCAAGGCACCAAACGGTCTTATGAACAGATGGAAACTGATGGGGAACGCCAGAATGCAACTGAGATCAGAGCATCCGTCGGGAAGATGATTGATGGAAT
TGGACGATTCTACATCCAAATGTGCACTGAACTTAAACTCAGTGATTATGAGGGGCGACTGATCCAGAAC
AGCTTAACAATAGAGAGAATGGTGCTCTCTGCTTTTGACGAAAGAAGGAATAAATATCTGGAAGAACATC
CCAGCGCGGGGAAGGATCCTAAGAAAACTGGAGGACCCATATACAAGAGAGTAGATAGAAAGTGGATGAG
GGAACTCGTCCTTTATGACAAAGAAGAAATAAGGCGAATCTGGCGCCAAGCCAATAATGGTGATGATGCA
ACAGCTGGTCTGACTCACATGATGATCTGGCATTCCAATTTGAATGATACAACATACCAGAGGACAAGAG
CTCTTGTTCGCACCGGCATGGATCCCAGGATGTGCTCTCTGATGCAGGGTTCGACTCTCCCTAGGAGGTC
TGGAGCTGCAGGCGCTGCAGTCAAAGGAGTTGGGACAATGGTGATGGAGTTGATAAGGATGATCAAACGT
GGGATCAATGATCGGAACTTCTGGAGAGGTGAAAATGGACGAAAAACAAGGAGTGCTTACGAGAGAATGT
GCAACATTCTCAAAGGAAAATTTCAAACAGCTGCACAAAGGGCAATGATGGATCAAGTGAGAGAAAGTCG
GAACCCAGGAAATGCTGAGATCGAAGATCTCATCTTTCTGGCACGGTCTGCACTCATATTGAGAGGGTCA
GTTGCTCACAAATCTTGTCTGCCCGCCTGTGTGTATGGACCTGCCGTAGCCAGTGGCTACGACTTCGAAA
AAGAGGGATACTCTTTAGTGGGAATAGACCCTTTCAAACTGCTTCAAAACAGCCAAGTATACAGCCTAAT
CAGACCGAACGAGAATCCAGCACACAAGAGTCAGCTGGTGTGGATGGCATGCAATTCTGCTGCATTTGAA
GATCTAAGAGTATTAAGCTTCATCAGAGGGACCAAAGTATCCCCAAGGGGGAAACTTTCCACTAGAGGAG
TACAAATTGCTTCAAATGAAAACATGGATGCTATGGAATCAAGTACTCTTGAACTGAGAAGCAGGTACTG
GGCCATAAGAACCAGAAGTGGAGGAAACACTAATCAACAGAGGGCCTCTGCAGGTCAAATCAGTGTGCAA
CCTGCATTTTCTGTGCAAAGAAACCTCCCATTTGACAAACCAACCATCATGGCAGCATTCACTGGGAATA
CAGAGGGAAGAACATCAGACATGAGGGCAGAAATTATAAGGATGATGGAAGGTGCAAAACCAGAAGAAAT
GTCCTTCCAGGGGCGGGGAGTCTTCGAGCTCTCGGACGAAAGGGCAGCGAACCCGATCGTGCCCTCTTTT
GACATGAGTAATGAAGGATCTTATTTCTTCGGAGACAATGCAGAGGAGTACGACAATTAA


## Process and analyze deep sequencing data

### Align paired-end reads

In [ ]:
def main():
    """Main body of script."""

    # Specify R1 and R2 read lengths and gene range of each subamplicon for dms_barcodedsubamplicons
    alignspecs = '1,252,34,32 253,501,38,30 502,750,30,36 751,1002,32,41 1003,1251,29,34 1252,1497,34,41'

    # Specify range of residues to consider
    sitesrange = ' '.join(['%s' %i for i in range(1,499)]) # range of residues to consider

    # Set R1 and R2 trim lengths for dms_barcodedsubamplicons
    R1trimlength = 200
    R2trimlength = 160

    # Group all barcodes with this many reads or more for dms_summarizealignments
    maxperbarcode = 4
 
   # Specify for which mutvir libraries to infer amino-acid preferences
    # I will not be running this since I didn't sequence the WT viruses
    # prefs_mutvir = {'replicate-1': ['mutvir', 'mutvir-MS-1A', 'mutvir-MS-1B', 'mutvir-MxA-1A', 'mutvir-MxA-1B'], \
                      #'replicate-2': ['mutvir', 'mutvir-MS', 'mutvir-MxA']}

    # Specify which mutvir libraries to use as control selection and actual selection to infer differential amino-acid preferences
    # I will be doing this, we want to infer differential AA preferences between folding environments
    controls = {'replicate-1': ['Sample1', 'Sample3', 'Sample5'], 'replicate-2': ['Sample9', 'Sample11', 'Sample13']}
    selections = {'replicate-1': ['Sample2', 'Sample4', 'Sample6'], 'replicate-2': ['Sample10', 'Sample12', 'Sample14']}

    # Directory path for FASTQ files for each replicate and amplicon
    FASTQ_path = '%s/FASTQ_files/' % basedir
    
    print "Found FASTQ_path"

    # Specify which pairs of mutvir libraries to get correlation of amino-acid preferences between replicate-1 and replicate-2 as 2-tuples
    # I dont think im running this since im not running prefs_mutvir
    #correlate_prefs_mutvir = ( ('mutvir', 'mutvir'), ('mutvir-MS-1A', 'mutvir-MS'), ('mutvir-MS-1B', 'mutvir-MS'), \
                              # ('mutvir-MxA-1A', 'mutvir-MxA'), ('mutvir-MxA-1B', 'mutvir-MxA'))

    # Specify location of preference files from older sequencing study. These will get compared to current sequencing study.
    # not relevant for me, i think.
    # old_basedir = '/fh/fast/bloom_j/grp/mapmuts/examples/2014Analysis_Influenza_NP_MxA'
    #files_old_prefs_mutvir = [('replicate-%s' %i, '%s/replicate_%s/preferences_mutvir/replicate_%s_mutvir_equilibriumpreferences.txt' % (old_basedir, i, i)) for i in ('1', '2')]
    #print files_old_prefs_mutvir

    # Toggle which scripts to run or skip:
    # however, this script is not smart enough to know if the existing output 
    # exists necessary to run the next strip exist - so use with caution!
    run_barcodedsubamplicons = True
    run_summarizealignments = False

# again, I'm not infering prefs
    #run_inferprefs = False
    #run_prefs_editsites = False
    #run_prefs_merge = False
    #run_prefs_correlate = False
    #run_prefs_logoplot = False

    run_inferdiffprefs = False
    run_diffprefs_editsites = False
    run_diffprefs_merge = False
    run_diffprefs_correlate = False
    run_diffprefs_logoplot = False
    #########################################
    #########################################
    # Now run dms_barcodedsubamplicons
    # Each sample is run its own subdirectory
    #########################################
    #########################################
    if run_barcodedsubamplicons:
        print "entering barcodedsubamplicons section of masterscript"

        processes = []

        # Store (r1files, r2files) in dictionary keyed by (replicate, sample)
        fastqfiles = {} 
        for replicate, sample in replicate_sample:
            r1files = sorted(glob.glob('%s/%s/%s/%s_1.fastq' % (FASTQ_path, replicate, sample, sample)))
            r2files = sorted(glob.glob('%s/%s/%s/%s_2.fastq' % (FASTQ_path, replicate, sample, sample)))
            assert r1files and r2files
            fastqfiles[(replicate, sample)] = (r1files, r2files)
            #print replicate, sample, r1files, r2files

            if not os.path.isdir(replicate):
                os.mkdir(replicate)
            if not os.path.isdir("%s/%s" % (replicate, sample)):
                os.mkdir("%s/%s" % (replicate, sample))

        # Make the barcoded alignments for each sample
        for replicate, sample in replicate_sample:
            subdir = '%s/%s' % (replicate, sample)
            alignprefix = '%s_' % sample
            (r1files, r2files) = fastqfiles[(replicate, sample)]
            commands = ['--R1trimlength %s' % R1trimlength, '--R2trimlength %s' % R2trimlength, alignprefix, refseq, ','.join(r1files), ','.join(r2files), alignspecs]

            # Add this makealignments.py call to the list of processes.
            # Use *list* to make a copy of *commands* since lists are mutable on subsequent runs
            # of making replicate-amplicon combinations.
            processes.append(multiprocessing.Process(target=RunScriptDms, args=(subdir, 'barcodedsubamplicons', 'dms_barcodedsubamplicons', list(commands), use_sbatch, 1)))
    

        # run ALL the barcodedsubamplicons processes
        RunProcesses(processes, nmultiruns=max_cpus)

        # end barcodedsubamplicons block of masterscript
    else:
        print "Skipping barcodedsubamplicons section of masterscript"
        
if __name__ == '__main__':
    main() # run the script

### Summarize alignments

In [ ]:
def main():
    """Main body of script."""

    # Group all barcodes with this many reads or more for dms_summarizealignments
    maxperbarcode = 4

    # Directory path for FASTQ files for each replicate and amplicon
    FASTQ_path = '%s/FASTQ_files/' % basedir


    # Run summarizealignments on individual samples
    for replicate, sample in replicate_sample:
        subdir = '%s/%s' % (replicate, sample)
        alignprefix_name = '%s_,%s' % (sample, sample)
        outprefix = 'alignmentsummary_'
        commands = ['--maxperbarcode %s' % maxperbarcode, outprefix, 'barcodedsubamplicons', alignprefix_name]            #print commands
#        RunScriptDms(subdir, 'summarizealignments', 'dms_summarizealignments', commands, False, 1)

    # Run summarizealignments on all samples from a replicate     
    for replicate in replicates:
        alignprefixes_names = []
        subdir = replicate
        outprefix = 'alignmentsummary_'
        for sample in replicate_sample_dict[replicate]:
            if replicate == 'replicate-1' and sample != 'DNA':   # only 7 alignments can be summarized currently
                alignprefix = '%s/%s_' % (sample, sample)
                alignprefixes_names.append("%s,%s" % (alignprefix, sample))
            if replicate != 'replicate-1':
                alignprefix = '%s/%s_' % (sample, sample)
                alignprefixes_names.append("%s,%s" % (alignprefix, sample))
        commands = ['--maxperbarcode %s' % maxperbarcode, outprefix, 'barcodedsubamplicons', ' '.join(alignprefixes_names)]
        print commands
        RunScriptDms(subdir, 'summarizealignments', 'dms_summarizealignments', commands, False, 1)
        
if __name__ == '__main__':
    main() # run the script

### Plot mutational and error rates, depth of coverage

In [ ]:
from IPython.display import Image, display

filepath = 'replicate-1/alignmentsummary_barcodes.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_depth.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutcounts_all.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutcounts_multi_nt.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutdepth.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_mutfreqs.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

filepath = 'replicate-1/alignmentsummary_reads.pdf'

png = filepath.rstrip('.pdf') + '.png'
!convert -density 192 -trim $filepath $png
display(Image(png, width=500))

### Calculate differential selection for each mutation and each site in NP

Run dms_diffselection on all perturbed proteostasis environments relative to basal conditions at 37 C for each of three replicates.

In [ ]:
#REPLICATE 1: -/+ 17AAG 37C
%cd preferences
!ls

!dms_diffselection Rep1-DMSO-37_counts.txt Rep1-17AAG-37_counts.txt 17AAG-rep1-37_